In [1]:
import torch, numpy as np, matplotlib.pyplot as plt
from torch import nn, from_numpy
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

class FeedforwardNN(nn.Module):
    def __init__(self):
        super(FeedforwardNN, self).__init__()
        self.nn = nn.Sequential(nn.Linear(1, 1))
    def forward(self, x):
        return self.nn(x)
    
def train(x, y, epoch=5, scaler=None):
    x, y = x.cuda().float(), y.cuda().float()
    model = FeedforwardNN().cuda()
    criterion = nn.MSELoss()
    optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.01)
    x_scaler, y_scaler = scaler
    for epoch in tqdm(range(epoch)):
        output = model(x)
        loss = criterion(output, y)
        optim.zero_grad()
        loss.backward()
        optim.step()

        ys = y_scaler.inverse_transform(model(from_numpy(np.linspace(0, 1, 100).reshape(-1, 1)).cuda().float()).cpu().detach().numpy())
        plt.plot(x_scaler.inverse_transform(np.linspace(0, 1, 100).reshape(-1, 1)), ys)
        plt.scatter(x_scaler.inverse_transform(x.cpu().detach().float()), y_scaler.inverse_transform(y.cpu().detach().float()))
        plt.title(loss.cpu().detach().numpy())
        plt.savefig('epoch'+str(epoch)+'.jpg')
        plt.clf()

    return model

x_scaler, y_scaler = MinMaxScaler(), MinMaxScaler()
height, weight = [163.3, 178, 172.5, 174.2, 170.7, 173.1, 177.6, 175, 173.3, 162.7], [46.64, 59.68, 54.11, 56.4, 49.44, 61.93, 63.67, 62.48, 59.64, 48.39]
x = from_numpy(x_scaler.fit_transform(np.array(height).reshape(-1, 1)))
y = from_numpy(y_scaler.fit_transform(np.array(weight).reshape(-1, 1)))
model = train(x, y, epoch=10, scaler = [x_scaler, y_scaler])

100%|██████████| 10/10 [00:01<00:00,  5.31it/s]


<Figure size 432x288 with 0 Axes>

In [2]:
# Taken from https://stackoverflow.com/questions/44947505/how-to-make-a-movie-out-of-images-in-python

import cv2
import os, glob
video_name = 'training.mp4'

images = glob.glob('*.jpg')
frame = cv2.imread(os.path.join(os.getcwd(), images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, 5, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(os.getcwd(), image)))

cv2.destroyAllWindows()
video.release()